# Documentation du Programme de Tri d'Objets avec Niryo One et Interface Graphique

Ce programme permet de contrôler un robot Niryo One pour trier des objets par couleur (vert, rouge, bleu) à l'aide d'une interface graphique (Tkinter). Il utilise la vision par ordinateur (OpenCV) pour détecter les objets dans un espace de travail défini et les déplacer vers des positions prédéfinies

## 1. Configuration Initiale

### Dépendances

In [ ]:
from pyniryo import *  # Contrôle du robot Niryo
import tkinter as tk    # Interface graphique
import cv2              # Traitement d'image
import numpy as np      # Manipulation de tableaux

### Définitions des Positions et Couleurs

- Positions de placement (colors) : Positions (en mètres) où le robot dépose les objets selon leur couleur

In [ ]:
colors = {
    "GREEN": PoseObject(0.0, 0.2, 0.2, 0.0, 1.57, 1.57),
    "RED": PoseObject(-0.15, 0.2, 0.2, 0.0, 1.57, 1.57),
    "BLUE": PoseObject(0.2, 0.2, 0.2, 0.0, 1.57, 1.57),
}

- Plages HSV (COLOR_HSV) : Intervalles HSV pour la détection des couleurs.

In [ ]:
COLOR_HSV = {
    "GREEN": ((35, 50, 50), (85, 255, 255)),
    "RED": ((0, 50, 50), (10, 255, 255)),
    "BLUE": ((100, 50, 50), (140, 255, 255)),
}

###  Connexion au Robot

- Adresse IP : <b>robot_ip_address = "192.168.0.101" </b>.

- Espace de travail : <b>workspace_name = "workspace2"</b> (doit être préalablement calibré via Niryo Studio).

- Pose d'observation : Position où le robot observe les objets.

In [ ]:
observation_pose = PoseObject(x=0.16, y=0.0, z=0.45, roll=0.0, pitch=1.57, yaw=0.0)

## 2. Fonctions Clés

### tri_objet()

- <b>Objectif </b>: Trier les objets détectés dans l'espace de travail.

- <b>Processus </b>:

    - Déplace le robot à la pose d'observation.

    - Utilise <b>robot.vision_pick()</b> pour détecter et saisir un objet.

    - Si un objet est trouvé, le place à la position correspondante.

    - Répète jusqu'à ce qu'aucun objet ne soit détecté.

In [ ]:
def tri_objet():
    while True:
        found_something = False  # Variable pour vérifier si un objet a été trouvé

        for couleur, position in colors.items():
            robot.move_pose(observation_pose)
            robot.wait(1)

            # Tentative de prise avec la caméra
            obj_found, shape_ret, color_ret = robot.vision_pick(workspace_name, color=couleur)
            print(f"Détection : Forme={shape_ret}, Couleur={color_ret}")

            if obj_found:
                found_something = True
                robot.place_from_pose(position)
                robot.move_pose(observation_pose)
            
            # Si aucun objet n'a été trouvé dans toute la boucle, on sort de la boucle
        if not found_something:
            print("Aucun objet détecté, arrêt du programme.")
            break


### bouton_video()

- Objectif : Afficher un flux vidéo en direct depuis la caméra du robot.

- Fonctionnement :

    - Capture et décompresse l'image via <b>robot.get_img_compressed()</b>.

    - Affiche l'image avec OpenCV (<b>cv2.imshow</b>).

In [ ]:
def bouton_video():
    # Moving to observation pose
    robot.move_pose(observation_pose)

    while True:
        # Getting image
        img_compressed = robot.get_img_compressed()
        # Uncompressing image
        img_raw = uncompress_image(img_compressed)
        # - Display
        # Showing images
        cv2.imshow("video",img_raw)
        key = cv2.waitKey(30)& 0xFF
        if key in [27, ord("q")]:  # Will break loop if the user press Escape or Q
            break
    cv2.destroyAllWindows

### fonction_image(couleur)

- <b>Objectif</b> : Afficher un masque filtrant une couleur spécifique (HSV).
  
- <b>Paramètre</b> : <b>couleur</b> ("GREEN", "RED", ou "BLUE").
  
- <b>Processus</b> :
  
    - Convertit l'image en espace HSV.
      
    - Applique un masque avec <b>cv2.inRange()</b> selon les plages HSV définies.

In [ ]:
def fonction_image(couleur):
# Moving to observation pose
    robot.move_pose(observation_pose)
    lower,upper=COLOR_HSV[couleur]

    while True:
        # Getting image
        img_compressed = robot.get_img_compressed()
        # Uncompressing image
        img_raw = uncompress_image(img_compressed)
        # conversion en HSV
        img_hsv=cv2.cvtColor(img_raw,cv2.COLOR_BGR2HSV)
        # FILTRAGE DE LA COULEUR
        mask=cv2.inRange(img_hsv,np.array(lower),np.array(upper))


        # Showing images
        cv2.imshow(f"Detection {couleur}",mask)
        key = cv2.waitKey(30)& 0xFF
        if key in [27, ord("q")]:  # Will break loop if the user press Escape or Q
            break

    cv2.destroyAllWindows


### Boutons de l'Interface Graphique

<b>Boutons</b> :
- <b>Video</b> : Lance le flux vidéo.

- <b>Filtres Vert/Rouge/Bleu</b> : Affiche le masque de la couleur sélectionnée.
    
- <b>Tri d'objet</b> : Démarre le tri automatique.

- <b>Quitter</b> : Ferme l'application.

## 3. Interface Graphique 

- Fenêtre Tkinter :

    - <b>Dimensions</b> : 400x300 pixels.
  
    - Boutons stylisés avec couleurs et polices spécifiques.
 
    - Organisés verticalement avec <b>pack(pady=5)</b>.

## 4. Utilisation

<b>Fonctionnalités</b> :

- Cliquez sur Bouton video pour voir le flux caméra.
  
- Utilisez les boutons de filtre pour tester la détection de couleur.
   
- Appuyez sur Tri d'objet pour lancer le tri automatique.

- Quitter arrête le programme.

## 5. Notes Importantes

- <b>Calibrage requis</b> : L'espace de travail (<b>workspace2</b>) doit être préalablement configuré via <b>Niryo Studio</b>.
  
- <b>Connexion réseau</b> : Vérifiez l'adresse IP du robot.
  
- <b>Sécurité</b> : Assurez-vous que la zone de travail est dégagée avant de démarrer le tri.

## 6. Dépannage

- <b>Erreurs de connexion</b> : Vérifiez l'IP et le câble Ethernet.

- <b>Détection incorrecte</b> : Ajustez les plages <b>HSV</b> dans <b>COLOR_HSV</b>.

- <b>Problèmes de mouvement</b> : Recalibrez le robot avec <b>robot.calibrate_auto()</b>.